In [1]:
import sys
import math
sys.setrecursionlimit(1500)  # set recursion up limit to a higher number

In [2]:
# this function is in charge of reading in the maze.txt and store it in 2D matrix
# input: a string of search file name
# output: a 2D matrix
def inputMaze(name="tinySearch.txt"):
    with open(name) as mazeTxt:
        maze = [list(line)[0:-1] for line in mazeTxt]
        rows = len(maze)
        maze[rows-1].append('%')
    return maze
# similar thing to above, slightly different because of the format diff between tiny and small, medium
def inputMaze1(name):
    with open(name) as mazeTxt:
        maze = [list(line)[0:-1] for line in mazeTxt]
        rows = len(maze)
        maze[rows-1].append('%')
    return maze

##### let's try to implement the new state representation and start with BFS

In [3]:
# store three searches into variable
tiny_search = inputMaze("tinySearch.txt")
small_search = inputMaze("smallSearch.txt")
medium_search = inputMaze("mediumSearch.txt")

In [1]:
# just checking out what the matrix look like
#cur_search = tiny_search
#print(cur_search)
#print(len(cur_search))
#for i in range(len(cur_search)):
#    print(len(cur_search[i]))

In [2]:
# input: 2D array
# output: start point coordinates, dot_positions and the number of dots
def findStartNDots(maze):
    dot_positions = set()   # dot_pos set will be used to hold dot coordinates in form of tuples
    dot_count = 0
    for i in range(len(maze)):
        for j in range(len(maze[0])):
            if maze[i][j] == 'P':
                start = (i,j)
            if maze[i][j] == '.':
                dot_count += 1
                dot_positions.add((i,j))
    return start, dot_positions, dot_count

#start, dot_pos, N = findStartNDots(tiny_search)
#print(start, dot_pos, N)

In [6]:
# this func checks whether cur_pos is on dot
# input: cor is a (x, y) tuple
# output: True or False
def isDot(cor, dot_pos):
    if cor in dot_pos:
        return True
    else:
        return False

In [7]:
# this function detects valid neighbors, generate new states based on them
# and return them
# input: current state, 2D array maze, height of maze, width of maez, dot positions
def multi_valid_neighbors(cur_state, maze, height, width, dot_pos):
    y = cur_state[2][0]
    x = cur_state[2][1]
    old_f = cur_state[0]
    cost = cur_state[1]+1
    cur_pos = (y, x)
    
    # make a list called valid_neighbors which will hold the states that we wanna return
    valid_neighbors = []
    
    # checking order: LEFT, DOWN, RIGHT, UP
    if (x-1) >= 0 and maze[y][x-1] != '%':
        new_pos = (y, x-1)
        cur_eaten = set(cur_state[3])
        if isDot(new_pos, dot_pos):
            cur_eaten.add(new_pos)
        #
        unvisited = allGoals.difference(cur_eaten)
        h = kruskal(EDGES, unvisited)[1]
        f = h + cost
        valid_neighbors.append( (f,cost,new_pos, frozenset(cur_eaten)) )
    if (y+1) < height and maze[y+1][x] != '%':
        new_pos = (y+1, x)
        cur_eaten = set(cur_state[3])
        if isDot(new_pos, dot_pos):
            cur_eaten.add(new_pos)
        unvisited = allGoals.difference(cur_eaten)
        h = kruskal(EDGES, unvisited)[1]
        f = h + cost
        valid_neighbors.append( (f,cost,new_pos, frozenset(cur_eaten)) )
    if (x+1) < width and maze[y][x+1] != '%':
        new_pos = (y, x+1)
        cur_eaten = set(cur_state[3])
        if isDot(new_pos, dot_pos):
            cur_eaten.add(new_pos)
        unvisited = allGoals.difference(cur_eaten)
        h = kruskal(EDGES, unvisited)[1]
        f = h + cost
        valid_neighbors.append( (f,cost,new_pos, frozenset(cur_eaten)) )
    if (y-1) >= 0 and maze[y-1][x] != '%':
        new_pos = (y-1, x)
        cur_eaten = set(cur_state[3])
        if isDot(new_pos, dot_pos):
            cur_eaten.add(new_pos)
        unvisited = allGoals.difference(cur_eaten)
        h = kruskal(EDGES, unvisited)[1]
        f = h + cost
        valid_neighbors.append( (f,cost,new_pos, frozenset(cur_eaten)) )          
    return valid_neighbors

# Let's try A* now

In [8]:
# input: current position, matrix, height, width
# output: a list of valid neighbors that should be explored later
def valid_neighbors(center, maze, height, width):
    y = center[0]
    x = center[1]
    valid_neighbors = []
    if (x+1) < width and maze[y][x+1] != '%':
        valid_neighbors.append((y,x+1))
    if (x-1) >= 0 and maze[y][x-1] != '%':
        valid_neighbors.append((y,x-1))
    if (y+1) < height and maze[y+1][x] != '%':
        valid_neighbors.append((y+1,x))
    if (y-1) >= 0 and maze[y-1][x] != '%':
        valid_neighbors.append((y-1,x))
    return valid_neighbors


# input: current position, goal position, maze, height, width
def BFSneighbors(current,goals,maze,height,width):
    #return the cost to each goal as a dictionary
    visited = height*width*[False]
    path = (height*width)*[None]
    cost = {}
    
    num = len(goals)
    
    ele_list = []
    ele_list.insert(0,current)
    visited[current[0]*width+current[1]] = True
    
    count = 0 
    c = {}
    
    c[current[0]*width+current[1]] = 0
    goal = None
    while ele_list:
        curr = ele_list.pop()
        count+=1
        if num == 0:
            break
        if curr in goals:
            num = num-1
        neighbors = valid_neighbors(curr,maze,height,width)
        for n in neighbors:
            if n!= None:
                pos = n[0]*width+n[1]
                if visited[pos] == False:
                    ele_list.insert(0,n)
                    visited[pos] = True
                    c[pos] = c[curr[0]*width+curr[1]]+1
                        
    for n in goals:
        ele = (n,current)
        cost[frozenset(ele)] = c[n[0]*width+n[1]]
        
    return [cost,count]

# this func renders a hashmap containing all the edges between any two dots
# input: 2D Array
# output: a dictionary containing all edges between two dots
def distance(search):
    [start, dot_pos, dot_count] = findStartNDots(search)
    height = len(search)
    width = len(search[0])
    dist  = {}
    for i in dot_pos:
            goals = list(dot_pos)
            index = goals.index(i)
            del goals[index]
            cost,count = BFSneighbors(i,goals,search,height,width)
            for n in cost:
                dist[n] = cost[n]
    return dist

In [9]:
# used to pick out the edges between unvisited dots
# return the hashmap which contains all unvisited edges
def pickEdges(unvisited, edges):
    # a hashmap which contains part of the all edges
    unvisitedEdges = {}
    for k, v in edges.items():
        unvisitedFlag = True
        for dot in k:
            if dot not in unvisited:
                unvisitedFlag = False
        if unvisitedFlag:
            unvisitedEdges[k] = v
    return unvisitedEdges

In [10]:
# this func checks if a loop will be formed is a new edge is added into 
# input: a list containing sets of vertices connected, a set containing two vertices of new edge
def checkLoop(trees, v_set):
    for tree in trees:
        if len(tree.intersection(v_set)) == 2:
            return False
        
    return True

In [11]:
# this func is used to merge subtrees after adding new edge
# input: a list containing sets of vertices connected, a set containing two vertices of new edge
# output: a list containing set of vertices connected
def merge(trees, v_set):
    intersect_tree = []
    for tree in trees:
        if tree.intersection(v_set):
            intersect_tree.append(tree)
            
    if len(intersect_tree) == 0:
        trees.append(set(v_set))    #we have to store a frozenset into the tree for now
    elif len(intersect_tree) == 1:
        intersect_tree[0].update(set(v_set))
    elif len(intersect_tree) == 2:
        intersect_tree[0].update(intersect_tree[1])
        trees.remove(intersect_tree[1])
    return trees

In [12]:
# kruskal algorithm, which is in charge of find the sum of length of all edges in MST
# input: all edges between two dots, and unvisited dots
# output: a dictionary containing all edges in the mst and the sum of length of all edges
def kruskal(edges, unvisited):
    
    trees = []    # a list of SET that stores the subtrees we have connected
    unvisitedEdges = pickEdges(unvisited, edges)
    mst = {}      # a DICT of edges
    
    while unvisitedEdges:
        shortest = float("inf")
        key = None
        
        # do a for loop to pick out the shortest edge
        for k, v in unvisitedEdges.items():
            if (v < shortest):
                shortest = v   # update v if that is smaller than shortest
                key = k
            #print(k, v)
        #print(trees)
        
        if checkLoop(trees, key):       # if no loop will be formed, add into mst
            #print("add one to mst")
            trees = merge(trees, key)
            mst[key] = unvisitedEdges[key]
            
        del unvisitedEdges[key]   # delete this edge after checking it
    
    length = 0
    for k, v in mst.items():
        length += v
    return mst, length

In [13]:
# this function returns distance to nearest dot
# input: position to explore, dots that have been eaten
# output: distance to nearest dot
def bfstoUnvisited(new_pos, cur_eaten):
    cur_uneaten = allGoals.difference(cur_eaten)
    costMap, count = BFSneighbors(new_pos, cur_uneaten, maze, height, width)
    lowestCost = min(costMap, costMap.get)
    return lowestCost

In [14]:
# the global variables
CUR_search = tiny_search

_, allGoals, _ = findStartNDots(CUR_search)
EDGES = distance(CUR_search)
print(allGoals)

# Astar search for 1.2
# input: 2D Array
# output: success(boolean), path(list), step count 
def Astar(search):
    height = len(search)
    width = len(search[0])
    
    # This hashmap store the edges with frozenset as key and a distance number as value
    dot_edges = distance(search)
    
    
    # initialize start_state and find needed info from maze
    start_pos, dot_pos, N = findStartNDots(search)
    unvisited = allGoals.difference(frozenset())
    h = kruskal(EDGES, unvisited)[1] 
    start_state = (h,0,start_pos, frozenset())
    
    # visited is a set that contain elements in the form of ((x,y), {visited dots' coordinates})
    visited = set()
    # make a hashmap to keep track of the route
    track = {}
    
    # add dumb ot the start
    dumb = None
    track[start_state] = dumb
    

    import heapq
    ele_list = []
    ele_list.insert(0, start_state)
    heapq.heapify(ele_list)
    
    
    count = 0 
    #mark start_state as visited
    visited.add(start_state)
    
    
    success = False
    final_state = start_state
    while ele_list:
        #print(ele_list)
        cur = heapq.heappop(ele_list)
        count += 1  #count the expansion     
            
        if len(cur[3]) == N:       # BASE CASE: if we have visited/eaten all the dots
            success = True
            final_state = cur
            break
        
        neighbors = multi_valid_neighbors(cur, search, height, width, dot_pos)
        for nei in neighbors:
            if nei!= None: 
                if nei not in visited:
                    heapq.heappush(ele_list,nei)
                    visited.add(nei)
                    track[nei] = cur
                    #old_cost = cur[1]
                    #nei[1] = old_cost+1
                    #nei[0] = fn(nei,nei[1])
          
        if count%1000 == 0: 
            print(count)
        
    path = []
    cur = final_state
    while cur:
        path.insert(0, cur[2])
        cur = track[cur]
    path.pop(0)
    return success, path, count

{(1, 22), (1, 25), (4, 18), (6, 1), (9, 19), (1, 14), (11, 28), (4, 13), (3, 28), (11, 13), (10, 1), (5, 28), (6, 17), (4, 1), (3, 5)}


In [15]:
success, path, count = Astar(tiny_search)
print(count)
print(path)
print(len(path))

1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
45913
[(1, 8), (1, 9), (1, 10), (1, 11), (1, 12), (1, 13), (1, 14), (1, 13), (1, 12), (1, 11), (1, 10), (1, 9), (2, 9), (3, 9), (3, 10), (3, 11), (3, 12), (4, 12), (4, 13), (4, 14), (4, 15), (5, 15), (6, 15), (6, 16), (6, 17), (6, 16), (6, 15), (5, 15), (4, 15), (3, 15), (3, 16), (2, 16), (1, 16), (1, 17), (1, 18), (2, 18), (3, 18), (4, 18), (3, 18), (2, 18), (1, 18), (1, 19), (1, 20), (1, 21), (1, 22), (1, 23), (1, 24), (1, 25), (1, 26), (1, 27), (1, 28), (2, 28), (3, 28), (2, 28), (1, 28), (1, 27), (1, 26), (1, 25), (1, 24), (1, 23), (2, 23), (3, 23), (3, 24), (4, 24), (5, 24), (5, 25), (5, 26), (5, 27), (5, 28), (5, 27), (5, 26), (5, 25), (5, 24), (5, 23), (6, 23), (7, 23), (8, 23), (9, 23), (10, 23), (11, 23), (11, 24), (11, 

In [ ]:
#save the output
count = 355256
path = [(9, 25), (9, 26), (9, 27), (9, 28), (9, 29), (9, 30), (8, 30), (7, 30), (7, 31), (7, 32), (7, 33), (8, 33), (8, 34), (8, 35), (8, 36), (9, 36), (9, 37), (9, 38), (9, 39), (9, 40), (9, 41), (9, 42), (9, 43), (9, 42), (9, 41), (9, 40), (8, 40), (7, 40), (7, 41), (7, 42), (7, 43), (6, 43), (6, 44), (6, 45), (7, 45), (8, 45), (9, 45), (9, 46), (9, 47), (10, 47), (11, 47), (11, 46), (11, 45), (11, 44), (11, 43), (11, 44), (11, 45), (11, 46), (11, 47), (10, 47), (9, 47), (9, 46), (9, 45), (8, 45), (7, 45), (6, 45), (6, 44), (6, 43), (7, 43), (7, 42), (7, 41), (6, 41), (5, 41), (5, 40), (4, 40), (3, 40), (3, 41), (3, 42), (3, 43), (4, 43), (4, 44), (4, 45), (3, 45), (3, 46), (3, 47), (2, 47), (1, 47), (1, 46), (1, 47), (2, 47), (3, 47), (3, 46), (3, 45), (4, 45), (4, 44), (4, 43), (3, 43), (3, 42), (3, 41), (3, 40), (3, 39), (3, 38), (3, 37), (3, 36), (4, 36), (3, 36), (3, 35), (3, 34), (2, 34), (2, 33), (2, 32), (1, 32), (2, 32), (3, 32), (4, 32), (5, 32), (5, 33), (5, 32), (5, 31), (5, 30), (5, 29), (5, 28), (5, 27), (5, 26), (5, 25), (5, 24), (5, 23), (4, 23), (4, 22), (4, 21), (4, 20), (4, 19), (4, 18), (4, 19), (4, 20), (5, 20), (6, 20), (6, 21), (6, 20), (6, 19), (6, 18), (6, 17), (6, 16), (6, 15), (5, 15), (4, 15), (4, 14), (3, 14), (3, 13), (4, 13), (4, 12), (4, 11), (3, 11), (3, 10), (3, 9), (2, 9), (1, 9), (1, 10), (1, 11), (1, 12), (1, 13), (1, 14), (1, 13), (1, 12), (1, 11), (1, 10), (1, 9), (2, 9), (3, 9), (3, 8), (4, 8), (4, 7), (4, 6), (3, 6), (4, 6), (4, 5), (4, 4), (3, 4), (2, 4), (1, 4), (1, 3), (1, 2), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (6, 2), (6, 3), (7, 3), (8, 3), (8, 2), (8, 1), (9, 1), (9, 2), (9, 3), (9, 4), (9, 5), (9, 6), (9, 7), (10, 7), (11, 7), (11, 6), (11, 5), (11, 6), (11, 7), (10, 7), (10, 8), (10, 9), (9, 9), (8, 9), (7, 9), (6, 9), (6, 10), (6, 11), (6, 12)]
cost = 207

In [7]:
            
count = 355256
p = [(9, 25), (9, 26), (9, 27), (9, 28), (9, 29), (9, 30), (8, 30), (7, 30), (7, 31), (7, 32), (7, 33), (8, 33), (8, 34), (8, 35), (8, 36), (9, 36), (9, 37), (9, 38), (9, 39), (9, 40), (9, 41), (9, 42), (9, 43), (9, 42), (9, 41), (9, 40), (8, 40), (7, 40), (7, 41), (7, 42), (7, 43), (6, 43), (6, 44), (6, 45), (7, 45), (8, 45), (9, 45), (9, 46), (9, 47), (10, 47), (11, 47), (11, 46), (11, 45), (11, 44), (11, 43), (11, 44), (11, 45), (11, 46), (11, 47), (10, 47), (9, 47), (9, 46), (9, 45), (8, 45), (7, 45), (6, 45), (6, 44), (6, 43), (7, 43), (7, 42), (7, 41), (6, 41), (5, 41), (5, 40), (4, 40), (3, 40), (3, 41), (3, 42), (3, 43), (4, 43), (4, 44), (4, 45), (3, 45), (3, 46), (3, 47), (2, 47), (1, 47), (1, 46), (1, 47), (2, 47), (3, 47), (3, 46), (3, 45), (4, 45), (4, 44), (4, 43), (3, 43), (3, 42), (3, 41), (3, 40), (3, 39), (3, 38), (3, 37), (3, 36), (4, 36), (3, 36), (3, 35), (3, 34), (2, 34), (2, 33), (2, 32), (1, 32), (2, 32), (3, 32), (4, 32), (5, 32), (5, 33), (5, 32), (5, 31), (5, 30), (5, 29), (5, 28), (5, 27), (5, 26), (5, 25), (5, 24), (5, 23), (4, 23), (4, 22), (4, 21), (4, 20), (4, 19), (4, 18), (4, 19), (4, 20), (5, 20), (6, 20), (6, 21), (6, 20), (6, 19), (6, 18), (6, 17), (6, 16), (6, 15), (5, 15), (4, 15), (4, 14), (3, 14), (3, 13), (4, 13), (4, 12), (4, 11), (3, 11), (3, 10), (3, 9), (2, 9), (1, 9), (1, 10), (1, 11), (1, 12), (1, 13), (1, 14), (1, 13), (1, 12), (1, 11), (1, 10), (1, 9), (2, 9), (3, 9), (3, 8), (4, 8), (4, 7), (4, 6), (3, 6), (4, 6), (4, 5), (4, 4), (3, 4), (2, 4), (1, 4), (1, 3), (1, 2), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (6, 2), (6, 3), (7, 3), (8, 3), (8, 2), (8, 1), (9, 1), (9, 2), (9, 3), (9, 4), (9, 5), (9, 6), (9, 7), (10, 7), (11, 7), (11, 6), (11, 5), (11, 6), (11, 7), (10, 7), (10, 8), (10, 9), (9, 9), (8, 9), (7, 9), (6, 9), (6, 10), (6, 11), (6, 12)]
cost = 207


filename = "1.2-med-Astar"
maze = inputMaze('mediumsearch.txt')
a,dot_pos,b = findStartNDots(maze)



order = []
for i in p:
    if isDot(i,dot_pos):
        order.append(i)
            
for i in range(len(order)):
    if i<10:
        maze[(order[i])[0]][(order[i])[1]] = chr(i+48)
    else:
        maze[(order[i])[0]][(order[i])[1]] = chr(i-10+97)

    #print(i)
f = open(filename,"w")
for i in range(len(maze)):
    for j in range(len(maze[0])):
        f.write(maze[i][j])
    f.write('\n')
f.write("cost is %d \n" %cost)
f.write("number of expanded nodes is %d \n"%count)
f.close()